In [1]:
import pickle
with open('ROMS_26yr_daydic.pkl', 'rb') as file:
    daydic = pickle.load(file)

with open('ROMS_26yr_eddies.pkl', 'rb') as file:
    eddies = pickle.load(file)

In [2]:
import netCDF4 as nc
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import time
from scipy.interpolate import interp1d
from scipy.ndimage import gaussian_filter

fname = '/srv/scratch/z3533156/26year_BRAN2020/outer_avg_01461.nc'
dataset = nc.Dataset(fname)
lon_rho  = dataset.variables['lon_rho'][:]
lon_rho = np.transpose(lon_rho, axes=(1, 0))
lat_rho  = dataset.variables['lat_rho'][:]
lat_rho = np.transpose(lat_rho, axes=(1, 0))
f  = dataset.variables['f'][:]
f = np.transpose(f, axes=(1, 0))
h = dataset.variables['h'][:]
h = np.transpose(h, axes=(1, 0))
z_r = np.load('/home/z5297792/UNSW-MRes/MRes/z_r/z_r.npy')
z_r = np.transpose(z_r, (1, 2, 0))
temp_ave = np.load('/srv/scratch/z5297792/Climatology/temp_ave.npy')
CurSpeed_ave = np.load('/srv/scratch/z5297792/Climatology/CurSpeed_ave.npy')
methods = ['VG', 'COW', 'LNAM']

import re
def extract_number(s):
    match = re.search(r'\d+', s)
    return int(match.group()) if match else None

## Shift Analysis

In [3]:
# with open('/srv/scratch/z5297792/Sample/frontier_sample_data.pkl', 'rb') as file:
#     frontier_sample_data = pickle.load(file)
# the_frontier_eddies = list(frontier_sample_data.keys())
# # Depths will be limited to COW depths
# d_df = pd.DataFrame(columns=['T' + str(t) for t in range(30)])
# for eddy in the_frontier_eddies:
#     row = []
#     for t in range(30):
#         df = frontier_sample_data[eddy]['T' + str(t)]['COW']['x']
#         row.append(df.isna().idxmax() if df.isna().any() else len(df))
#     d_df.loc[eddy] = row

# eddy_sub_data = {}

# methods = ['VG', 'COW', 'LNAM']
# for eddy in the_frontier_eddies:
    
#     sub_data = {}
#     for t in range(30):
        
#         sub_data['T'+str(t)] = {}
    
#         for method in methods:
    
#             sub_data['T'+str(t)][method] = {}
    
#             df = frontier_sample_data[eddy]['T'+str(t)][method].iloc[:d_df.loc[eddy]['T'+str(t)]]
#             df = df.drop('TD', axis=1)
            
#             # Interpolate to Density intervals
#             rho = frontier_sample_data[eddy]['T'+str(t)]['rho']
#             X, Y = np.meshgrid(frontier_sample_data[eddy]['T'+str(t)]['x'],
#                                frontier_sample_data[eddy]['T'+str(t)]['y'])
#             points = np.column_stack((X.ravel(), Y.ravel()))
#             rhoc = []
#             for k, (xc, yc) in enumerate(zip(df['x'], df['y'])):
#                 values = rho[:, :, k].ravel()
#                 target_point = np.array([[xc, yc]])
#                 rhoc.append(griddata(points, values, target_point, method='linear')[0])
#             RHO_INT = 0.1  # Average d\rho/dz = 0.01
#             new_rho = np.arange(np.floor(np.nanmin(rhoc) / RHO_INT) * RHO_INT,
#                             np.ceil(np.nanmax(rhoc) / RHO_INT) * RHO_INT + RHO_INT,
#                             RHO_INT)
#             new_rho = [round(value, 2) for value in new_rho]
#             rho_data = pd.DataFrame({
#                 'x': np.interp(new_rho, rhoc, df['x']),
#                 'y': np.interp(new_rho, rhoc, df['y']),
#                 'Depth': np.interp(new_rho, rhoc, df['Depth']),
#                 'rho': new_rho
#             })
#             x0 = rho_data.iloc[0]['x']
#             y0 = rho_data.iloc[0]['y']
#             rho_data['x'] = rho_data['x'] - x0
#             rho_data['y'] = rho_data['y'] - y0
#             rho_data['TD'] = np.hypot(rho_data['x'], rho_data['y'])
            
#             # Interpolate to Depth intervals
#             DEPTH_INT = 10
#             new_depth = np.arange(0, -np.floor(df['Depth'].min()/DEPTH_INT) * DEPTH_INT + DEPTH_INT, DEPTH_INT)
#             depth_data = pd.DataFrame({
#                 'x': np.interp(new_depth, -df['Depth'], df['x']),
#                 'y': np.interp(new_depth, -df['Depth'], df['y']),
#                 'Depth': -new_depth,
#                 'rho': np.interp(new_depth, -df['Depth'], rhoc),
#             })
#             x0 = depth_data.iloc[0]['x']
#             y0 = depth_data.iloc[0]['y']
#             depth_data['x'] = depth_data['x'] - x0
#             depth_data['y'] = depth_data['y'] - y0
#             depth_data['TD'] = np.hypot(depth_data['x'], depth_data['y'])
        
#             sub_data['T'+str(t)][method]['rho_data'] = rho_data
#             sub_data['T'+str(t)][method]['depth_data'] = depth_data
#         if t % 10 == 0:
#             print(t)
#     eddy_sub_data[eddy] = sub_data
#     print(eddy)
#     del sub_data

with open('/srv/scratch/z5297792/Sample/eddy_sub_data.pkl', 'rb') as file:
    eddy_sub_data = pickle.load(file)
the_frontier_eddies = list(eddy_sub_data.keys())

In [8]:
the_frontier_eddies

['Eddy790', 'Eddy110', 'Eddy134', 'Eddy775']

In [15]:
depth_data_dic = {}
for eddy in the_frontier_eddies:
    depth_data_dic[eddy] = {}
    for method in methods:
        
        depth_dfs = []
        for t in range(30):
            for col in ['x', 'y', 'TD']:
                depth_data = eddy_sub_data[eddy][f'T{t}'][method]['depth_data'][['Depth', col]]
                depth_data.columns = ['Depth', f'{col}{t}']
                depth_dfs.append(depth_data.set_index('Depth'))
        depth_df = pd.concat(depth_dfs, axis=1).sort_index(ascending=False)
        depth_data_dic[eddy][method] = depth_df

rho_data_dic = {}
for eddy in the_frontier_eddies:
    rho_data_dic[eddy] ={}
    for method in methods:
        rho_dfs = []
        for t in range(30):
            for col in ['x', 'y', 'TD']:
                rho_data = eddy_sub_data[eddy][f'T{t}'][method]['rho_data'][['rho', col]]
                rho_data.columns = ['rho', f'{col}{t}']
                rho_dfs.append(rho_data.set_index('rho'))
        rho_df = pd.concat(rho_dfs, axis=1).sort_index()
        rho_data_dic[eddy][method] = rho_df


In [16]:
rho_data_dic[eddy][method]

,x0,y0,TD0,x1,y1,TD1,x2,y2,TD2,x3,...,TD26,x27,y27,TD27,x28,y28,TD28,x29,y29,TD29
rho,,,,,,,,,,,,,,,,,,,,,
24.7,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24.8,3000.0,0.000000,3000.000000,3000.000000,124.774813,3002.593671,3436.340892,6436.340892,7296.226614,-1083.046076,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24.9,3000.0,3000.000000,4242.640687,6000.000000,6499.326095,8845.407830,6000.000000,15000.000000,16155.494421,-3952.380555,...,3722.262059,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
25.0,3000.0,4244.091814,5197.337331,4973.052957,7973.052957,9396.852088,3000.000000,12000.000000,12369.316877,-10332.742586,...,21840.329668,-9000.000000,0.000000,9000.000000,-6000.0,0.0,6000.0,-6000.0,3000.0,6708.203932
25.1,3000.0,6000.000000,6708.203932,3414.412894,6828.825787,7634.859333,3000.000000,12000.000000,12369.316877,-11083.560837,...,24738.633754,-12000.000000,0.000000,12000.000000,-6000.0,0.0,6000.0,-6000.0,0.0,6000.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12198.839841,5154.596435,8577.298218,10006.993011,NaN,NaN,NaN,NaN,NaN,NaN
32.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12275.007863,5577.731326,8788.865663,10409.382613,NaN,NaN,NaN,NaN,NaN,NaN
32.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12368.784805,6000.000000,9000.000000,10816.653826,NaN,NaN,NaN,NaN,NaN,NaN


## Optimal 3D Shift Computation

### Solve the Problem

$$
\huge x' = x\cos\theta - y\sin\theta + a
$$
$$
\huge y' = x\sin\theta + y\cos\theta + b
$$
\
$$
\begin{aligned}
\huge R(a, b, \theta) & \huge = \sum_i \left[x_i'(t_1) - x_i(t_2)\right]^2 + \left[y_i'(t_1) - y_i(t_2)\right]^2 \\
                      & \huge = \sum_i \left[x_i(t_1)\cos\theta - y_i(t_1)\sin\theta + a - x_i(t_2) \right]^2 + \left[x_i(t_1)\sin\theta + y_i(t_1)\cos\theta + b - y_i(t_2)\right]^2
\end{aligned}
$$